In [1]:
# Notebook for Workshop: DecisionTrees for Heart Failure Prediction
# Canvas URL: https://hiof.instructure.com/courses/8923/assignments/40855

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

In [2]:
# Load the dataset and show first 5 rows

dataset = pd.read_csv('../data/heart_failure/heart_failure_clinical_records_dataset.csv')
dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [3]:
# Split into 60/20/20. 60% for training, 20% validation, and 20% for final testing

trainval = dataset.groupby('DEATH_EVENT', group_keys=False).apply(lambda x: x.sample(frac=0.8))
test = dataset.drop(trainval.index)
# 75% of 80% is 60% of total
train = trainval.groupby('DEATH_EVENT', group_keys=False).apply(lambda x: x.sample(frac=0.75))
validation = trainval.drop(train.index)

In [4]:
# Separate input features from targets in all partitions

train_features = train.drop('DEATH_EVENT', axis=1)
train_targets = train['DEATH_EVENT']

validation_features = validation.drop('DEATH_EVENT', axis=1)
validation_targets = validation['DEATH_EVENT']

test_features = test.drop('DEATH_EVENT', axis=1)
test_targets = test.DEATH_EVENT

In [5]:
# Train a classifier with default parameters

model1 = DecisionTreeClassifier()
model1.fit(train_features, train_targets)

DecisionTreeClassifier()

In [6]:
# Evaluate the performance of the classifier using the default metrics for classifiers, accuracy

model1.score(validation_features, validation_targets)

0.7627118644067796

In [7]:
# Evaluate the performance using accuracy, precision, and recall 
def evaluate(model, final_eval=False):

    # Store the predictions

    train_predictions = model.predict(train_features)
    validation_predictions = model.predict(validation_features)

    # Print the training and validation scores

    print(f'Training Accuracy: {accuracy_score(train_targets, train_predictions)}')
    print(f'Training Precision: {precision_score(train_targets, train_predictions)}')
    print(f'Training Precision: {recall_score(train_targets, train_predictions)}')
    print(f'Validation Accuracy: {accuracy_score(validation_targets, validation_predictions)}')
    print(f'Validation Precision: {precision_score(validation_targets, validation_predictions)}')
    print(f'Validation Precision: {recall_score(validation_targets, validation_predictions)}')
    
    # Evaluate and print the final test scores if final_eval is True
    
    if final_eval:
        test_predictions = model.predict(test_features)

        print(f'Test Accuracy: {accuracy_score(test_targets, test_predictions)}')
        print(f'Test Precision: {precision_score(test_targets, test_predictions)}')
        print(f'Test Precision: {recall_score(test_targets, test_predictions)}')


In [8]:
# Evaluate our model using our custom evaluate function

evaluate(model1)

Training Accuracy: 1.0
Training Precision: 1.0
Training Precision: 1.0
Validation Accuracy: 0.7627118644067796
Validation Precision: 0.6
Validation Precision: 0.7894736842105263


In [9]:
# Similar reports can be created using the builtin classification_report, but 
# I recommend creating your own to have full control:

classification_report(validation_targets, model1.predict(validation_features))

'              precision    recall  f1-score   support\n\n           0       0.88      0.75      0.81        40\n           1       0.60      0.79      0.68        19\n\n    accuracy                           0.76        59\n   macro avg       0.74      0.77      0.75        59\nweighted avg       0.79      0.76      0.77        59\n'

In [10]:
# Train a new classifier with max_depth set to 2 i.e. the produced decision tree will 
# never be deeper than 2 levels. This is a somewhat arbitraty choice of both parameter and 
# value. You should try as many combinations as you see fit to achieve the best possible 
# classifier.

model2 = DecisionTreeClassifier(max_depth=2)
model2.fit(train_features, train_targets)
evaluate(model2)

Training Accuracy: 0.8555555555555555
Training Precision: 0.9
Training Precision: 0.6206896551724138
Validation Accuracy: 0.8135593220338984
Validation Precision: 0.7222222222222222
Validation Precision: 0.6842105263157895


In [11]:
# Do final evaluation with the best models only

evaluate(model1, final_eval=True)
evaluate(model2, final_eval=True)

Training Accuracy: 1.0
Training Precision: 1.0
Training Precision: 1.0
Validation Accuracy: 0.7627118644067796
Validation Precision: 0.6
Validation Precision: 0.7894736842105263
Test Accuracy: 0.8333333333333334
Test Precision: 0.8
Test Precision: 0.631578947368421
Training Accuracy: 0.8555555555555555
Training Precision: 0.9
Training Precision: 0.6206896551724138
Validation Accuracy: 0.8135593220338984
Validation Precision: 0.7222222222222222
Validation Precision: 0.6842105263157895
Test Accuracy: 0.8333333333333334
Test Precision: 0.8461538461538461
Test Precision: 0.5789473684210527
